# <center> 2차 크롤링 - <원티드> </center>

##### 링크들 들어가서 각각 정보 수집, 로봇 제어

In [1]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib


In [2]:
options = webdriver.ChromeOptions()

url = "https://www.wanted.co.kr/jobsfeed"
driver = webdriver.Chrome(service=Service("../../driver/chromedriver"),
                          options=options)

driver.get(url)

In [3]:
driver.get(url)

In [3]:
nav_element = driver.find_element(By.CLASS_NAME, "Aside_asideList__3WNyu")
search_element = nav_element.find_element(By.CLASS_NAME, "SvgIcon_SvgIcon__root__8vwon")

search_element.click()

In [4]:
search_doc = driver.find_element(By.CLASS_NAME, "SearchInput_SearchInput__gySrv")
search_doc.send_keys('로봇 제어')
search_doc.send_keys(Keys.ENTER)


In [5]:
wait = WebDriverWait(driver, 10)
position_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(), "포지션")]')))
position_tab.click()

In [9]:
simple_list = []
wait = WebDriverWait(driver, 10)  

simple_name_elements = []  

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    new_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')
    simple_name_elements.extend(new_elements)

   # StaleElementReferenceException을 처리하기 위해 엘리먼트 수의 변경을 확인
    try:
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_location__2EOr5')) > len(simple_name_elements))
    except TimeoutException:
        break

    if len(simple_name_elements) >= 20:
        break


simple_names = [element.text.strip() for element in simple_name_elements]

print(simple_names)
print(len(simple_names))

['', '', '', '', '', '', '', '', '서울 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '인천 · 한국', '서울 · 한국', '서울 · 한국', '인천 · 한국', '경기 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '서울 · 한국', '경기 · 한국']
24


In [10]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
# 기업이름 요소 초기 로드
company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
num_elements_before_scroll = len(company_name_elements)
while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break
    # 현재 로드된 회사명 요소들 가져오기
    company_name_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_companyName__vZMqJ')
    num_elements_before_scroll = len(company_name_elements)
    if num_elements_before_scroll >= 20:
        break
company_names = []
href_lists = []
href_elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-attribute-id="position__click"]')
# 가져온 데이터 중에서 조건에 맞는 것들만 필터링
for i, element in enumerate(company_name_elements):
    simple_name = simple_names[min(i, len(simple_names) - 1)]
    company_name_text = element.text.strip()
    # simple_name에 '서울 · 한국' 또는 '경기 · 한국'이 포함된 경우에만 추가
    if '서울 · 한국' in simple_name or '경기 · 한국' in simple_name:
        company_names.append(company_name_text)
        href_value = href_elements[i].get_attribute('href')
        href_lists.append(href_value)

In [11]:
#직무(담당업무 열)가져오기
job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')
num_elements_before_scroll = len(company_name_elements)
while True:
    # 페이지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try:
        # 새로운 회사명 요소들이 로드될 때까지 대기 (타임아웃 10초)
        wait.until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')) > num_elements_before_scroll)
    except TimeoutException as ex:
        break
    job_title_elements = driver.find_elements(By.CLASS_NAME, 'JobCard_title__ddkwM')
    num_elements_before_scroll = len(company_name_elements)
    if num_elements_before_scroll >= 20:
        break
job_titles = []
for i, element in enumerate(job_title_elements):
    # simple_names 리스트에서 현재 인덱스에 해당하는 값 가져오기
    simple_name = simple_names[min(i, len(simple_names)-1)]
    # 만약 값이 '서울'이나 '경기'로 시작하면 추가
    if simple_name.startswith('서울') or simple_name.startswith('경기'):
        job_titles.append(job_title_elements[i].text.strip())  # 수정된 부분

In [12]:

# 기업주소 리스트 초기화
address_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기업 주소를 포함한 <section> 요소 찾기
        address_div = soup.find('section', class_='JobWorkPlace_className__ra6rp')

        if address_div:
            # <section> 내부의 주소 정보를 찾아서 리스트에 추가
            address_items = address_div.find_all('span', class_='body')
            address = [address_item.text.strip() for address_item in address_items]

            # URL과 해당하는 주소 리스트를 한 리스트에 추가
            address_list.append(address)
            break  # 주소를 찾았으므로 루프 종료

        # 스크롤을 크게 아래로 내림 (300픽셀씩)
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)  # 스크롤 후 충분한 시간 대기



In [13]:
# address_list에 있는 각 기업주소에 대해 처리
for i in range(len(address_list)):
    # 첫 번째 요소를 제외한 나머지 주소로 갱신
    address_list[i] = address_list[i][1:]

# 리스트 출력
for address in address_list:
    print(f'{address}\n')

print(len(address_list))

['서울특별시 종로구 대학로 60 동마루빌딩 2층']

['경기도 성남시 분당구 분당수서로 477, HD현대그룹 R&D센터 10F']

['서울 금천구 디지털로10길 78 가산테라타워 10층']

['서울특별시 종로구 대학로 60 동마루빌딩 2층']

['서울시 서초구 서초대로 396, 7층 (서초동, 강남빌딩)']

['금천구']

['경기도 성남시 분당구 판교로']

['금천구']

['서울 금천구 디지털로10길 78 가산테라타워 10층']

['서울특별시 성동구 성수이로22길 37, 아크벨리지식산업센터 6층']

['서울시 성동구 아차산로17길 48']

['성수동 연무장길 89']

['용산구 서빙고로 17, 센트럴파크타워 1705호']

['경기도 시흥시 경기과기대로 239, 제이케이로보틱스']

14


In [14]:
#연봉
salary_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    # 페이지 이동
    driver.get(url)


    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 기술 스택을 포함한 <div> 요소 찾기
        salary_div = soup.find('div', class_='Tags_tagsClass__mvehZ')

        if salary_div:
            salary_items = salary_div.find_all('li')
            first_li = salary_items[0] if salary_items else None

            if first_li and "연봉" in first_li.text:
                salaries = [salary_item.text.strip().replace("#", "") for salary_item in salary_items]
                salary_list.append(salaries[0])
                break
            else:
                salary_list.append(["-"])
                break


     
# 리스트 출력
for i in salary_list:
    print(f'{i}\n')

print(len(salary_list))

['-']

['-']

['-']

['-']

['-']

연봉업계평균이상

연봉상위6~10%

연봉업계평균이상

['-']

['-']

['-']

['-']

['-']

['-']

14


In [15]:
# 자격요건 텍스트를 저장할 리스트 초기화
qualifications_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "자격요건" 부분을 추출
        qualifications_header = soup.find('h3', string='자격요건')

        if qualifications_header:
            # "자격요건" 부분이 포함된 <span> 요소 찾기
            qualifications_span = qualifications_header.find_next('p').find('span')

            if qualifications_span:
                # <span> 요소에서 텍스트 추출
                qualifications_text = qualifications_span.get_text(strip=True)
                # 리스트에 자격요건 추가
                qualifications_list.append(qualifications_text)
                break  # 자격요건을 찾았으므로 루프 종료
            else:
                print('자격요건이 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for qualifications in qualifications_list:
    print(f'{qualifications}\n')

print(len(qualifications_list))

• Robot Engineering / Mechanical Engineering 에 대한 전반적인 이해• Linux 기반 C/C++, Python Programming 경험• 응용 프로그램 분야에 대한 포괄적 이해• 해외여행에 결격사유가 없는자

• 석사 이상, 경력 3년 이상• 서보모터 제어 개발 경험자• Embedded system 소프트웨어 경험자

ㆍ관련 경력 3년이상ㆍ실내 Navigation 분야 상용화 제품 개발 경력자(저널 논문도 가능)ㆍGlobal path algorithm 개발 경력자(A* 등등)ㆍLocal path planning 개발 경력자(DWA, RRT, 등등)ㆍROS, C++, python 개발 능통ㆍ탁월한 의사 소통 및 대인 관계 기술을 바탕으로 팀 중심 환경에서 작업할 수 있는 능력ㆍ비즈니스 회화 능력ㆍGit 경험자

• 경력 무관• Robot Engineering / Mechanical Engineering 에 대한 전반적인 이해• Linux 기반 C/C++, Python Programming 경험• 응용 프로그램 분야에 대한 포괄적 이해• 해외여행에 결격사유가 없는자

• 로보틱스/메카트로닉스/기계공학/전기전자공학 등 이 · 공 계열 학석사 이상 또는그에 상응하는 경력의 소지자• Robot Control SW / Embedded SW /Device 통신/ Robot 설계 및 제어• Embedded System 핸들링 능력(Data Processing/CAN/SPI/LAN/GPIO)• RTOS 기반 메인 컨트롤 SW 개발 및 구현 능력(Multi-task 기반의 Motion Control)• MATLAB, C++, C#, 3D CAD, ROS 등을 활용한 Robot 개발 및 제품 구현 능력• 논문이나 기술관련 문서를 이해하고 이를 바탕으로 System 구현이 가능한 능력• Git을 통한 프로젝트 협업 능력• 구성원들과의 협업이 가능한 의사소통 능력과 이를 통한 기술적 문제 해결 능력( 자격요건과 업무 경험은 신입/경력에 따라 or

In [16]:
#담당업무
work_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "주요업무" 부분을 추출
        work_header = soup.find('h3', string='주요업무')

        if work_header:
            work_span = work_header.find_next('p').find('span')

            if work_span:
                # <span> 요소에서 텍스트 추출
                work_text = work_span.get_text(strip=True)
                work_list.append(work_text)
                break  
            else:
                print('주요업무가 포함된 <span> 요소를 찾을 수 없습니다.')
        else:
            # 스크롤을 계속해서 아래로 내림
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)

for work in work_list:
    print(f'{work}\n')

print(len(work_list))

• 의료 로봇 응용 프로그램 개발 업무• 의료 로봇 모션 제어• C, C++, Python 등을 활용한 로봇 시나리오 구현• 의료 로봇 응용 프로그램 아키텍쳐 설계 및 구현

• 서보 제어 알고리즘 분석 및 응용• 다축 서보모터 제어 알고리즘 개발• 모터 제어 Embedded S/W 개발

ㆍ지도에서 효율적이고 안전한 global path planning 알고리즘 개발ㆍ복잡하고 역동적인 환경에서 작동할 수 있는 local path planning 알고리즘 개발＜전형절차＞서류검토 ＞ 1차면접 ＞ 임원면접 ＞ 최종합격 (절차는 변동될 수 있습니다.)＜유의사항＞허위사실이 발견될 경우 채용이 취소될 수 있습니다.

• 의료 로봇 응용 프로그램 개발 업무• 의료 로봇 모션 제어• C, C++, Python 등을 활용한 로봇 시나리오 구현• 의료 로봇 응용 프로그램 아키텍쳐 설계 및 구현

• 보행 로봇 제어 시스템 설계 및 SW 개발• RTOS 기반 로봇 제어 SW 개발• Embedded System 엔지니어링• 보행 로봇 Locomotion 개발 및 Field Test

• 로봇 셋업 및 안정화 업무 (80%)• 로봇 소프트웨어 및 GUI Application 개발 (20%)

- 로봇 제어 프로그래밍 (수직 다관절 로봇/물류 이송 로봇 등)- 모터 제어 알고리즘 설계 및 개발- ROS기반 각종 센서 장치 드라이버 개발- 물류 자동화 시스템 구축 및 개발[근무조건]• 근무형태 : 정규직• 근무지 : 판교 테크노밸리(경기도 성남시)• 근무시간 : 주5일, 8시 ~ 17시• 급여 : 회사내규에 따름(면접 후 결정)[전형절차]서류전형  - 1차면접 - 온라인 인성검사 - 2차면접(채용검진) -처우협의(레퍼런스 체크)- 최종합격※ 2차면접 및 레퍼런스 체크는 상황에 따라 생략될 수 있습니다.[접수방법]• 접수 기간: 채용시 마감※ 이력서 접수 순서대로 면접을 진행하며 우수인재 확보 시 마감• 이력서: 자유양식 (희망연봉 필수기재)[기타사항]• 입사지원 서류에 허위사실이 발견

In [21]:
#우대사항
preferential_list = []

# href_lists에 있는 각 링크에 대해 처리
for url in href_lists:
    driver.get(url)

    try:
        # 기다리기를 설정하여 페이지 로딩을 기다림
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    except TimeoutException:
        print('페이지 로딩이 제때 완료되지 않았습니다.')

    # 스크롤을 아래로 내리는 동작을 반복
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # "우대사항" 부분을 추출
        preferential_header = soup.find('h3', string='우대사항')

        if preferential_header:
            preferential_span = preferential_header.find_next('p').find('span')

            if preferential_span:
                # <span> 요소에서 텍스트 추출
                preferential_text = preferential_span.get_text(strip=True)
                preferential_list.append(preferential_text)
                break  
            else:
                break 
        else:
            # 우대사항이 페이지에 없으면 리스트에 공백 추가
            preferential_list.append('')
            break

for preferential in preferential_list:
    print(f'{preferential}\n')

print(len(preferential_list))

• 해당 분야 석사/ 박사 학위• Robot framework 개발 및 툴 사용 경험• Robot Hardware 관련 경험• 맥슨 모터 제어 관련 경험• ROS2 개발 관련 경험• Gazebo, NVIDIA ISSAC platform for Robotics, Webots  등 로봇 시뮬레이션 관련 경험

• Verilog, VHDL 개발 경험자 우대

ㆍNavigation 관련 분야 석사 혹은 박사 학위 소유자ㆍISO13482 인증 경험자 우대ㆍAMR or 자율주행로봇 (라인트레이서 제외)개발 경험자ㆍROS2 가능자ㆍ열정적인 모습으로 팀원들에게 동기 부여를 줄 수 있는 분

• 해당 분야 석사/ 박사 학위• Robot framework 개발 및 툴 사용 경험• Robot Hardware 관련 경험• 맥슨 모터 제어 관련 경험• ROS2 개발 관련 경험• Gazebo, NVIDIA ISSAC platform for Robotics, Webots  등 로봇 시뮬레이션 관련 경험

• 로보틱스/메카트로닉스/기계공학/전기전자공학 등 이 · 공 계열 박사 이상또는 5년 이상의 실무 경력• 로봇 제어/설계/ RTOS기반 제어 SW/Firmware/Device 통신/Locomotion(보행로봇) 및Field Test 경험• 로봇에 대한 Control(position, velocity, impedance control)에 관한 이해와 구현 경험• 로봇 Kinematics/Robotics 및 구동 메커니즘에 대한 높은 이해• Vision/Lidar 등의 센서 퓨전을 활용한 환경인식 및 경로 생성• Slack, Confluence, JIRA 등 협업 툴 경험

• Robot Engineering 경험이 있으신 분• C++, C# 프로그래밍에 익숙하신 분• WPF, Winform GUI 개발 경험자

- Robotics 분야 개발에 능통한 자- Embedded System 구성 및 소프트웨어 개발 경력자- ROS/ROS2 이용 로봇 SW 개발 경험 보유자

• Robot Eng

In [22]:
driver.quit()

In [23]:
# 같은 인덱스끼리 합치기(자격요건 + 우대사항 + 담당업무)
required_list = [qual + pref + work for qual, pref, work in zip(qualifications_list, preferential_list, work_list)]
for item in required_list:
    print(item)


• Robot Engineering / Mechanical Engineering 에 대한 전반적인 이해• Linux 기반 C/C++, Python Programming 경험• 응용 프로그램 분야에 대한 포괄적 이해• 해외여행에 결격사유가 없는자• 해당 분야 석사/ 박사 학위• Robot framework 개발 및 툴 사용 경험• Robot Hardware 관련 경험• 맥슨 모터 제어 관련 경험• ROS2 개발 관련 경험• Gazebo, NVIDIA ISSAC platform for Robotics, Webots  등 로봇 시뮬레이션 관련 경험• 의료 로봇 응용 프로그램 개발 업무• 의료 로봇 모션 제어• C, C++, Python 등을 활용한 로봇 시나리오 구현• 의료 로봇 응용 프로그램 아키텍쳐 설계 및 구현
• 석사 이상, 경력 3년 이상• 서보모터 제어 개발 경험자• Embedded system 소프트웨어 경험자• Verilog, VHDL 개발 경험자 우대• 서보 제어 알고리즘 분석 및 응용• 다축 서보모터 제어 알고리즘 개발• 모터 제어 Embedded S/W 개발
ㆍ관련 경력 3년이상ㆍ실내 Navigation 분야 상용화 제품 개발 경력자(저널 논문도 가능)ㆍGlobal path algorithm 개발 경력자(A* 등등)ㆍLocal path planning 개발 경력자(DWA, RRT, 등등)ㆍROS, C++, python 개발 능통ㆍ탁월한 의사 소통 및 대인 관계 기술을 바탕으로 팀 중심 환경에서 작업할 수 있는 능력ㆍ비즈니스 회화 능력ㆍGit 경험자ㆍNavigation 관련 분야 석사 혹은 박사 학위 소유자ㆍISO13482 인증 경험자 우대ㆍAMR or 자율주행로봇 (라인트레이서 제외)개발 경험자ㆍROS2 가능자ㆍ열정적인 모습으로 팀원들에게 동기 부여를 줄 수 있는 분ㆍ지도에서 효율적이고 안전한 global path planning 알고리즘 개발ㆍ복잡하고 역동적인 환경에서 작동할 수 있는 local path planning 알고리즘 개발＜전형절차＞

In [24]:
keyword_list = ["로봇제어"] * len(company_names)

print(len(keyword_list))
print(keyword_list)

14
['로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어', '로봇제어']


In [25]:

import pandas as pd
# 데이터프레임 생성
data = {
    '기업이름': company_names,
    '연봉': salary_list,
    '기업주소': address_list,
    '키워드': keyword_list,
    '담당업무': job_titles,
    '필요역량': required_list
}
df = pd.DataFrame(data)
# 데이터프레임 출력
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,메디인테크,[-],[서울특별시 종로구 대학로 60 동마루빌딩 2층],로봇제어,"[전문연구요원 가능] C/C++, Python 로봇 제어 개발자 (신입)",• Robot Engineering / Mechanical Engineering 에...
1,에이치디현대로보틱스,[-],"[경기도 성남시 분당구 분당수서로 477, HD현대그룹 R&D센터 10F]",로봇제어,로봇 시스템 서보제어 개발,"• 석사 이상, 경력 3년 이상• 서보모터 제어 개발 경험자• Embedded sy..."
2,알지티,[-],[서울 금천구 디지털로10길 78 가산테라타워 10층],로봇제어,모바일 로봇 제어 SW 개발자 (navigation),ㆍ관련 경력 3년이상ㆍ실내 Navigation 분야 상용화 제품 개발 경력자(저널 ...
3,메디인테크,[-],[서울특별시 종로구 대학로 60 동마루빌딩 2층],로봇제어,"[전문연구요원 가능] C/C++, Python 로봇 제어 개발자",• 경력 무관• Robot Engineering / Mechanical Engine...
4,알지에이아이엔씨(RGA Inc.),[-],"[서울시 서초구 서초대로 396, 7층 (서초동, 강남빌딩)]",로봇제어,로봇 제어 엔지니어,• 로보틱스/메카트로닉스/기계공학/전기전자공학 등 이 · 공 계열 학석사 이상 또는...


In [26]:
# • 문자를 ,으로 바꾸기
df['담당업무'] = df['담당업무'].str.replace('•', ',')
df['필요역량'] = df['필요역량'].str.replace('•', ',')

# 작은 따옴표 및 괄호 제거
df['연봉'] = df['연봉'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df['기업주소'] = df['기업주소'].astype(str).str.replace('[', '').str.replace(']', '').str.strip("'")
df.head()

,기업이름,연봉,기업주소,키워드,담당업무,필요역량
0,메디인테크,-,서울특별시 종로구 대학로 60 동마루빌딩 2층,로봇제어,"[전문연구요원 가능] C/C++, Python 로봇 제어 개발자 (신입)",", Robot Engineering / Mechanical Engineering 에..."
1,에이치디현대로보틱스,-,"경기도 성남시 분당구 분당수서로 477, HD현대그룹 R&D센터 10F",로봇제어,로봇 시스템 서보제어 개발,", 석사 이상, 경력 3년 이상, 서보모터 제어 개발 경험자, Embedded sy..."
2,알지티,-,서울 금천구 디지털로10길 78 가산테라타워 10층,로봇제어,모바일 로봇 제어 SW 개발자 (navigation),ㆍ관련 경력 3년이상ㆍ실내 Navigation 분야 상용화 제품 개발 경력자(저널 ...
3,메디인테크,-,서울특별시 종로구 대학로 60 동마루빌딩 2층,로봇제어,"[전문연구요원 가능] C/C++, Python 로봇 제어 개발자",", 경력 무관, Robot Engineering / Mechanical Engine..."
4,알지에이아이엔씨(RGA Inc.),-,"서울시 서초구 서초대로 396, 7층 (서초동, 강남빌딩)",로봇제어,로봇 제어 엔지니어,", 로보틱스/메카트로닉스/기계공학/전기전자공학 등 이 · 공 계열 학석사 이상 또는..."


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기업이름    14 non-null     object
 1   연봉      14 non-null     object
 2   기업주소    14 non-null     object
 3   키워드     14 non-null     object
 4   담당업무    14 non-null     object
 5   필요역량    14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


In [28]:
import re

# 경력 정보 추출을 위한 정규표현식
career_pattern = re.compile(r'(\d+년.*?)')

# DataFrame의 각 행에 대해 경력 정보 추출 및 새로운 열에 추가
df['고용형태'] = df['필요역량'].apply(lambda x: career_pattern.findall(x)[0] if career_pattern.findall(x) else '-')

# 결과 출력
print(df[['고용형태']])

   고용형태
0     -
1    3년
2    3년
3     -
4    5년
5    2년
6   10년
7     -
8    3년
9    8년
10    -
11   5년
12   3년
13    -


In [29]:
df = df[['기업이름','고용형태','연봉','기업주소','키워드','담당업무','필요역량' ]]

In [30]:
df.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,메디인테크,-,-,서울특별시 종로구 대학로 60 동마루빌딩 2층,로봇제어,"[전문연구요원 가능] C/C++, Python 로봇 제어 개발자 (신입)",", Robot Engineering / Mechanical Engineering 에..."
1,에이치디현대로보틱스,3년,-,"경기도 성남시 분당구 분당수서로 477, HD현대그룹 R&D센터 10F",로봇제어,로봇 시스템 서보제어 개발,", 석사 이상, 경력 3년 이상, 서보모터 제어 개발 경험자, Embedded sy..."
2,알지티,3년,-,서울 금천구 디지털로10길 78 가산테라타워 10층,로봇제어,모바일 로봇 제어 SW 개발자 (navigation),ㆍ관련 경력 3년이상ㆍ실내 Navigation 분야 상용화 제품 개발 경력자(저널 ...
3,메디인테크,-,-,서울특별시 종로구 대학로 60 동마루빌딩 2층,로봇제어,"[전문연구요원 가능] C/C++, Python 로봇 제어 개발자",", 경력 무관, Robot Engineering / Mechanical Engine..."
4,알지에이아이엔씨(RGA Inc.),5년,-,"서울시 서초구 서초대로 396, 7층 (서초동, 강남빌딩)",로봇제어,로봇 제어 엔지니어,", 로보틱스/메카트로닉스/기계공학/전기전자공학 등 이 · 공 계열 학석사 이상 또는..."


In [31]:
import csv

df.to_csv('output2.csv', index=False)